# Table of content
1. Import required libraries
2. Setup *plotly* library for using it offline
3. Load Strava Data using the *stravaio* library
4. Prepare Strava Data for plotting
5. Make the Infographics with Plotly

## 1. Import required libraries

In [ ]:
# Autoreload the libraries, useful for when new libraries are added on the fly
%load_ext autoreload
%autoreload 2
import os # will be used for reading environment variables
from stravaio import StravaIO # will be used for accessing Strava data
import maya # will be used for processing datetime information
from pprint import pprint # just a handy printing function
import pandas as pd # if you know this, you are qualified at least for the data analitics job 
import numpy as np # the mother of all data processing in Python

## 2. Setup plotly offline

The main purpose of the plotly library is to generate plots for the web using python. The library, however, allows using it in the offline mode, inside the jupyter notebook.

In [ ]:
from plotly.offline import iplot # the offline plot renderring function (as opposed to plot)
import plotly.graph_objs as go # object that generates various types of traces e.g. Scatter, Bar etc.
from plotly import tools # utility tools, we will use a make_subplot to generate a grid layout of plots
# Make plots to render in the notebook
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

## 3. Load Strava Data

To access your strava data programmatically you will need an **access token**. Strava recently changed their policy and accessing a token with activity level permission become a bit cumbersome. For such cases I've created a project that allows to lounch a local server to get the personal token. Go to the [strava-oauth](https://github.com/sladkovm/strava-oauth) for more info.

In [ ]:
access_token = os.getenv('STRAVA_ACCESS_TOKEN', None)
assert access_token is not None

To access the strava data I've written a *stravaio* library. It's purpose is to be as declarative as possible and the user should never have a feeling that he is working with a web service. In the center of the *stravaio* workflow is the *StravaIO* object that exposes all required functions to access *athlete*, *activities* and *streams* data.

In [ ]:
client = StravaIO(access_token)

StravaIO object directly exposes [Strava Swagger API interfaces](https://developers.strava.com/docs/reference/)

In [ ]:
client.__dict__

The StravaIO exposes a number of methods that allow direct access to Strava data (essentially these are the wrappers around the api interfaces that simplify life)

In [ ]:
for m in dir(client):
    if not m.startswith('_'):
        pprint(m)

### Get athlete information
Only information about the logged in athlete could be accessed. The access_token unambiguosly encodes the information about the athlete.

In [ ]:
athlete = client.get_logged_in_athlete()

In [ ]:
pprint(f"""
Name: {athlete.api_response.firstname}, 
Last Name: {athlete.api_response.lastname}, 
FTP: {athlete.api_response.ftp}, 
""")

### Get the athletes activities

The *after* parameter takes the date in human readable format - you can even tell it "Last year". The function returns a list of SummaryActivity.

In [ ]:
activities = client.get_logged_in_athlete_activities(after='20180101')

In [ ]:
type(activities[0])

### Get the streams corresponding to each activity

1. For each activity we will request a stream from Strava API if the activity contains the power data.
2. If the stream is already stored locally, the local copy (pandas.DataFrame) will be returned. Otherwise the Stream object will be requested from the API. The requested from the API stream will be stored locally for future reuse.
3. We will also start populating a list that will be used for making infographics:

```
[{
'activity': SummaryActivity,
'streams': pd.DataFrame(streams)
},
...
]
```

In [ ]:
to_plot = []
for a in activities:
    if (a.device_watts): # check if the activity has the power data
        pprint(f'{maya.parse(a.start_date).iso8601()}:, {a.name}, {a.start_latlng}, {a.trainer}, {a.type}')
        s = client.get_activity_streams(a.id, athlete.api_response.id)
        if isinstance(s, pd.DataFrame): # check whether the stream was loaded from the local copy
            _s = s
        else: # Streams were loaded from the API, will be stored locally first
            s.store_locally()
            _s = s.to_dict()
        to_plot.append({
            'activity': a,
            'streams': s
        })

In [ ]:
pprint(f"Total number of processed activities: {len(to_plot)}")

## 4. Prepare the data for plotting

For plotting we will need the following data:
1. Best 5s, 1min, 5 min and 20 min power from each ride
2. If the best power constitute a PR, the number will colored in Strava Orange
3. Smooth power data with 10s rolling mean - to make it prettier on the plot
4. Color coding of the each power point whether it is above (orange) or below (white) the FTP value

In [ ]:
def power_max(watts):
    """Returns a list of Power Bests for the ride"""
    pm = [
        watts.rolling(5).mean().max(),
        watts.rolling(60).mean().max(),
        watts.rolling(5*60).mean().max(),
        watts.rolling(20*60).mean().max()
    ]
    rv = [int(p) if p is not np.NAN else 0 for p in pm]
    return rv


def power_color(new, old):
    """Return orange if the new values is a PR"""
    if new > old:
        return "#fc4c02"
    else:
        return "#FFF"

In [ ]:
power_old = [0,0,0,0]
for d in to_plot:
    
    _power_max = power_max(d['streams']['watts'])
    d.update({'power_max': _power_max})
    
    _power_color = [power_color(pn, po) for pn, po in zip(_power_max, power_old)]
    d.update({'power_color': _power_color})
    
    power_old = [pn if pn>po else po for pn, po in zip(_power_max, power_old)]
    
    _streams = d['streams']
    _streams['watts_smooth'] = _streams['watts'].rolling(10, min_periods=1).mean()
    
    _streams['color'] = pd.Series(np.arange(len(_streams['watts'])))
    _streams['color'].loc[_streams['watts']>=athlete.api_response.ftp] = "#fc4c02"
    _streams['color'].loc[_streams['watts']<athlete.api_response.ftp] = "#FFF"
    d.update({'streams': _streams})
    
    d.update({'date': maya.parse(d['activity'].start_date).slang_date()})

In [ ]:
to_plot[0]['power_max']

## 5. Plot

In [ ]:
n_plots = len(to_plot)
MODE = 'markers'
# n_plots = 5
specs = [[{}, {}, {}] for i in range(n_plots)]
fig = tools.make_subplots(rows=n_plots, cols=3,
                          specs=specs,
                          shared_xaxes=True,
                          shared_yaxes=True,
                          vertical_spacing=0.0001)


yaxis_settings = dict(showline=False, zeroline=False, showgrid=False,
                                                  showticklabels=False, zerolinewidth=0)
xaxis_settings1 = dict(showline=False, zeroline=False, showgrid=False,
                                                  showticklabels=False,
                      domain=[0, 0.1])

xaxis_settings2 = dict(showline=False, zeroline=False, showgrid=False,
                                                  showticklabels=True,
                    tickvals=[3600, 2*3600, 3*3600],
                    ticktext=['1h' ,'2h', '3h'],
                      domain=[0.1, 0.8])

xaxis_settings3 = dict(showline=False, zeroline=False, showgrid=False,
                                                  showticklabels=True,
                      domain=[0.8, 1.0])


_=1
for d in to_plot:
    if _ < (n_plots+1):
        # Date and name
        trace = go.Bar({
            "x":[0],
            "y":[1],
            "text": [d['date']],
            "textposition": "outside",
            "textfont": {"color": "#FFF"},
            "marker": {"color": "#111",
                       "line": {
                           "width": 1,
                           "color": "#fc4c02"
                       }
                      }
        })
        fig.append_trace(trace, _, 1)
        
        # Scatter
        trace = go.Scattergl({
            "x": d['streams']['time'],
            "y": d['streams']['watts_smooth'],
            "mode": MODE,
            "marker": {"color": d['streams']['color'],
                       "size": 1},
            "textfont": {"color": "#FFF"},
        })
        fig.append_trace(trace, _, 2)
        
        # Bar
        trace = go.Bar({
            "x": ["5 sec", "1 min", "5 min", "20 min"],
            "y": [1,1,1,1],
            "text": d['power_max'],
            "textposition": "outside",
            "textfont": {"color": d['power_color']},
            "marker": {"color": '#111',
                       "line": {
                           "width": 1,
                           "color": "#fc4c02"
                       }
                      }
        })
        fig.append_trace(trace, _, 3)
        _+=1
        if _ > 1:
            fig['layout'].update({f'yaxis{2*_ - 1}': yaxis_settings})
            fig['layout'].update({f'yaxis{2*_-2}': yaxis_settings})
        else:
            fig['layout'].update({'yaxis': yaxis_settings})
            fig['layout'].update({'yaxis2': yaxis_settings})
    else:
        break
        
fig['layout'].update(title=f"""
{athlete.api_response.firstname} {athlete.api_response.lastname} Year in Strava 2018: stravaio, strava-oauth
""",
                     font=dict(family="Courier New", color="#FFF"),
                     height=1600, width=1000,
                     showlegend=False, paper_bgcolor='#111', plot_bgcolor='#111',
                     yaxis=yaxis_settings,
                     xaxis=xaxis_settings1,
                     xaxis2=xaxis_settings2,
                     xaxis3=xaxis_settings3)

# fig['layout'].update({'yaxis2':yaxis_settings})

In [ ]:
iplot(fig)